<a href="https://colab.research.google.com/github/Jollyhrothgar/banjo_liberation/blob/main/Banjo_Liberation_One_Notebook_%5BMike_Copy%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [37]:
import copy
import json

# Note Class

In [38]:
class Note:
  ''' Note class handles note/octave object, transposition, and frequency
    calculation
  '''

  NOTES = ['c','c#','d','d#','e','f','f#','g','g#','a','a#','b']

  def __init__(self, note):
    ''' Instantiate note, examples:
      Note('C')    # C4 (middle C)
      Note(0)    # C at octave 0
      Note(1)    # C-sharp at octave 0
      Note(12)     # C at octave 1
      Note('C2')   # C at octave 2
      Note('Db')   # D-flat at octave 4
      Note('D#3')  # D-sharp at octave 3
      Note(('G', 5)) # G note at octave 5
    '''
    if isinstance(note, str):
      self.index = Note.index_from_string(note)
    elif isinstance(note, tuple):
      self.index = Note.index_from_string(note[0]) + 12 * note[1]
    elif isinstance(note, Note):
      self.index = note.index
    else:
      self.index = int(note)
  
  def get_note_name(self):
    return f"{self.note}{self.octave}"

  def __repr__(self):
    return f"Note('{self.note}{self.octave}')"

  def __lt__(self, other):
    return self.index < other.index

  def __gt__(self, other):
    return self.index > other.index

  def __eq__(self, other):
    return self.index == other.index

  def __float__(self):
    return self.frequency()

  @property
  def note(self):
    ''' note name property
    '''
    return Note.NOTES[self.index % 12]

  @property
  def octave(self):
    ''' octave number property
    '''
    return int(self.index / 12)

  @classmethod
  def index_from_string(cls, note):
    ''' Get index number from note string
    '''
    octave = 4
    note = note.strip().lower()
    if note[-1].isdigit():
      note, octave = note[:-1], int(note[-1])
    if len(note) > 1:
      note = cls.normalize(note)
    return cls.NOTES.index(note) + 12 * octave

  @classmethod
  def normalize(cls, note):
    ''' Translate accidentals and normalize flats to sharps
      For example E#->F, F##->G, Db->C#
    '''
    index = cls.NOTES.index(note[0].lower())
    for accidental in note[1:]:
      if accidental == '#':
        index += 1
      elif accidental == 'b':
        index -= 1
    return cls.NOTES[index % 12]

  def at_octave(self, octave):
    ''' Return new instance of note at given octave
    '''
    return Note((self.index % 12) + 12 * octave)

  def transpose(self, halfsteps):
    ''' Return transposed note by halfstep delta
    '''
    return Note(self.index + halfsteps)

  def frequency(self):
    ''' Return frequency of note
    '''
    return 16.35159783128741 * 2.0 ** (float(self.index) / 12.0)

# Banjo Chord Generation

In [114]:
ROLL_DICT = {
  'square_roll': (3, 2, 5, 1),
  'triple_forward_square': (4, 3, 2, 3, 2, 1, 4, 3, 2, 3, 2, 1, 5, 2, 3, 1),
  'forward_backward': (3, 2, 1, 2 , 1, 2, 3, 1),
}

BANJO_TUNING = {
  1: Note('d5'),
  2: Note('b4'),
  3: Note('g4'),
  4: Note('d4'),
  5: Note('g5')
}

MOVEABLE_CHORDS = {
  'gamma_g4': (0, 0, 0, 0, 0),
    
}

CHORD_DICT = {
  'gamma_g1': (0, 0, 0, 0, 0),
  'beta_c1': (2, 1, 0, 2, 0),
  'beta_d1': (4, 3, 2, 4, 0),
  'alpha_g1': (5, 3, 4, 5, 0),
  'alpha_c1': (10, 8, 9, 10, 0),
  'alpha_d1': (12, 10, 11 ,12, 0),
  'beta_c2': (14, 13, 12, 14, 0),
  'beta_g2': (9, 8, 7, 9, 0),
  'beta_d2': (16, 15, 14, 16, 0),
}


def render_roll(roll, note_length):
  """
  Given a roll (an array of Notes) and a note_length (in seconds), render the wave-form that the
  audio synthesizer needs to make some noise.
  Args:
    roll: an array of Notes
    note_length: how long to play each note.
  Returns:
    rendered notes (a wave-form that is played with an audio synthesizer)
  """
  return [Hit(note=note, length=note_length).render() for note in roll]


def play_roll(roll):
  """Play each note in a rendered roll sequentially"""
  for note in roll:
    play(note)


def fret_strings(chord):
  """
  Fret strings pulls BANJO_TUNING from the global namespace and frets the strings
  according to the supplied chord.
  chord: a tuple which maps the depressed fret to the banjo string, according to the schema:
    chord = (
      first string fret,
      second string fret,
      third string fret,
      fourth string fret,
      fifth string fret
    )
  Args:
    chord: a tuple of depressed frets
  Returns:
    A dictionary of banjo strings mapped to the note after the chord has been applied.
  """
  global BANJO_TUNING
  fretted_strings = dict(BANJO_TUNING)

  for str_idx, fret in enumerate(chord):
    banjo_string = str_idx + 1
    fretted_strings[banjo_string] = fretted_strings[banjo_string].transpose(fret)
  return fretted_strings


def make_alpha_minor(chord):
  """Transform an alpha major to alpha minor chord
  
  (2, 0, 1, 2, 0)
  """
  return tuple([string - 1 if idx == 2 else string for idx, string in enumerate(chord)])


def make_alpha_seven(chord):
  pass

def make_alpha_minor_seven(chord):
  pass

def make_beta_minor(chord):
  """Transform a beta to a beta minor chord
  
  (2, 1, 0, 2, 0)
  """
  return tuple([string - 1 if idx in (0, 3) else string for idx, string in enumerate(chord)])


def make_beta_seven(chord):
  pass

def make_beta_minor_seven(chord):
  pass

def make_gamma_minor(chord):
  """ Transform a gamma to gamma minor
  (0, 0, 0, 0, 0)
  """
  return tuple([string - 1 if idx == 1 else string for idx, string in enumerate(chord)])

def make_gamma_seven(chord):
  pass

def make_gamma_minor_seven(chord):
  pass

def generate_moveable_chords(file_name=None):
  """
  Given the first location of moveable chords represented as a tuple:
    chord = (
      first string fret,
      second string fret,
      third string fret,
      fourth string fret,
      fifth string fret
    )
  
  Generate all moveable chords on the banjo. Optionally, each chord can be transformed with a
  chord transformer (but the base shapes could also just be specified hardcoded).
  #TODO(mike): Do something with file_name
  Args:
    file_name: A string that contains the full path to the file to serialize the generated
    chords.
  Returns:
    A dictionary mapping the normalized Note name (representing the pitch of the chord) to the
    frets that are depressed to play the chord.
  """
  first_alpha = (2, 0, 1, 2, 0) # e
  first_beta  = (2, 1, 0, 2, 0) # c
  first_gamma = (0, 0, 0, 0, 0) # g
  moveable_chords = {}

  for fret_position in range(0, 23):
    # These are all tuples that map strings to fret positions:
    # E.g. alpha = (
    #          first string fret position,
    #          second string fret position,
    #          third string fret position,
    #          fourth string fret position,
    #          fifth string fret position,
    #        )
    # Add the fret offset to each base chord, and don't add an offset to the fifth string.
    alpha = tuple([banjo_string + fret_position if string_idx != 4 else banjo_string for string_idx, banjo_string in enumerate(first_alpha)])
    alpha_minor = make_alpha_minor(alpha)
    beta = tuple([banjo_string + fret_position if string_idx != 4 else banjo_string for string_idx, banjo_string in enumerate(first_beta)])
    beta_minor = make_beta_minor(beta)
    gamma = tuple([banjo_string + fret_position if string_idx != 4 else banjo_string for string_idx, banjo_string in enumerate(first_gamma)])
    gamma_minor = make_gamma_minor(gamma)

    fretted_alpha = fret_strings(alpha)
    fretted_alpha_minor = fret_strings(alpha_minor)
    fretted_beta = fret_strings(beta)
    fretted_beta_minor = fret_strings(beta_minor)
    fretted_gamma = fret_strings(gamma)
    fretted_gamma_minor = fret_strings(gamma_minor)
    
    # Make sure we're not playing frets that don't exist
    if any([fret > 22 for fret in alpha]) or any([fret < 0 for fret in alpha]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['alpha_' + fretted_alpha[1].get_note_name()] = alpha

    if any([fret > 22 for fret in beta]) or any([fret < 0 for fret in beta]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['beta_' + fretted_beta[2].get_note_name()] = beta 

    if any([fret > 22 for fret in beta_minor]) or any([fret < 0 for fret in beta_minor]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['beta_minor_' + fretted_beta_minor[2].get_note_name()] = beta_minor 

    if any([fret > 22 for fret in gamma]) or any([fret < 0 for fret in gamma]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['gamma_' + fretted_gamma[3].get_note_name()] = gamma
    
    if any([fret > 22 for fret in gamma_minor]) or any([fret < 0 for fret in gamma_minor]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['gamma_minor_' + fretted_gamma_minor[3].get_note_name()] = gamma_minor

    if any([fret > 22 for fret in alpha_minor]) or any([fret < 0 for fret in alpha_minor]):
      pass # Chord contains frets that are greater than 22 or less than 0.
    else:
      moveable_chords['alpha_minor_' + fretted_alpha_minor[1].get_note_name()] = alpha_minor
  
  return moveable_chords


def make_roll(strings, roll_pattern):
  """
  Given a set of strings and a roll pattern, return the strings to be plucked in order of
  plucking.
  Args:
    strings: A banjo strings dictionary.
    roll_pattern: A tuple which determins what order to pluck the strings.
  Returns:
    An array of Notes.
  """
  return [strings[pluck] for pluck in roll_pattern]

# Tests

In [115]:
def test_chords():
  chord_1 = fret_strings(chord=CHORD_DICT['alpha_g1'])
  chord_2 = fret_strings(chord=CHORD_DICT['alpha_c1'])
  chord_3 = fret_strings(chord=CHORD_DICT['alpha_d1'])
  chord_4 = fret_strings(chord=CHORD_DICT['alpha_g1'])

  for name, chord in generate_moveable_chords(file_name=None).items():
    print(name, chord)

def test_note():
  test_note = Note('g4')
  print(test_note.get_note_name())


In [116]:
test_chords()

alpha_e5 (2, 0, 1, 2, 0)
beta_c5 (2, 1, 0, 2, 0)
beta_minor_c5 (1, 1, 0, 1, 0)
gamma_g4 (0, 0, 0, 0, 0)
alpha_minor_e5 (2, 0, 0, 2, 0)
alpha_f5 (3, 1, 2, 3, 0)
beta_c#5 (3, 2, 1, 3, 0)
beta_minor_c#5 (2, 2, 1, 2, 0)
gamma_g#4 (1, 1, 1, 1, 0)
gamma_minor_g#4 (1, 0, 1, 1, 0)
alpha_minor_f5 (3, 1, 1, 3, 0)
alpha_f#5 (4, 2, 3, 4, 0)
beta_d5 (4, 3, 2, 4, 0)
beta_minor_d5 (3, 3, 2, 3, 0)
gamma_a4 (2, 2, 2, 2, 0)
gamma_minor_a4 (2, 1, 2, 2, 0)
alpha_minor_f#5 (4, 2, 2, 4, 0)
alpha_g5 (5, 3, 4, 5, 0)
beta_d#5 (5, 4, 3, 5, 0)
beta_minor_d#5 (4, 4, 3, 4, 0)
gamma_a#4 (3, 3, 3, 3, 0)
gamma_minor_a#4 (3, 2, 3, 3, 0)
alpha_minor_g5 (5, 3, 3, 5, 0)
alpha_g#5 (6, 4, 5, 6, 0)
beta_e5 (6, 5, 4, 6, 0)
beta_minor_e5 (5, 5, 4, 5, 0)
gamma_b4 (4, 4, 4, 4, 0)
gamma_minor_b4 (4, 3, 4, 4, 0)
alpha_minor_g#5 (6, 4, 4, 6, 0)
alpha_a5 (7, 5, 6, 7, 0)
beta_f5 (7, 6, 5, 7, 0)
beta_minor_f5 (6, 6, 5, 6, 0)
gamma_c5 (5, 5, 5, 5, 0)
gamma_minor_c5 (5, 4, 5, 5, 0)
alpha_minor_a5 (7, 5, 5, 7, 0)
alpha_a#5 (8, 6, 7, 8, 

In [42]:
test_note()

g4


# Goal for today

* Four measures of sixteenth notes
* Measure 1: G, Measure 2: C, Measure 3: G, Measure 4: D
* Export to tabledit

In [127]:
def roll_on_chord(roll_pattern, chord):
  """ Assume that a roll is 16h notes, an eight note roll
  repeated twice.
  Args:
    roll_pattern: a string such as T1I1M3 - this means Thumb on first string
      index on first string, middle on third string.
    chord: a tuple, such as alpha g5, e.g. (5, 3, 4, 5, 0) # fret
                                           (1, 2, 3, 4, 5) # string number
                                           (0, 1, 2, 3, 4) # tuple index
  Returns A time sequenced list of notes and durations and frets
    List[
      {
        'fret': 3 ,
        'time': 2,
        'string': convert roll pattern to string.
      }
    ]
  """
  string_index_lookup = {
      '1': 0,
      '2': 1,
      '3': 2,
      '4': 3,
      '5': 4
  }

  # roll_pattern T4I3M2 -> T4 I3 M2
  n = 2
  roll = [roll_pattern[i:i+n] for i in range(0, len(roll_pattern), n)]

  output = []

  for finger in roll:  
    string_number = finger[1]
    string_idx = string_index_lookup[string_number]

    # Get the right fret
    fret = chord[string_idx]

    # append the string, duration, and fret to the output object. 
    output.append(
        {
            'fret': fret,
            'time': 2,
            'string': int(string_number)
        }
    )
  return output
    

In [128]:
def prototab_to_ascii(proto_tab):
  """Takes prototab and returns ascii.
  
  Assume that no notes are played at the same time. Use the duration
  of each note to fill out each string in ascii with either a played
  note or '-' characters. Grow the ascii from left to right.
  """
                   #  1 ,    2,     3,     4,    5
  ascii_strings = [  '|',   '|',   '|',   '|',  '|']
  first_string = 0
  second_string = 1
  third_string = 2
  fourth_string = 3
  fifth_string = 4

  # trinary operator
  # value = option1 if condition true else option2

  for note in proto_tab:
    ascii_strings[first_string]  += str(note['fret']).ljust(note['time'], '-') if note['string'] == 1 else '-'.ljust(note['time'], '-')
    ascii_strings[second_string] += str(note['fret']).ljust(note['time'], '-') if note['string'] == 2 else '-'.ljust(note['time'], '-')
    ascii_strings[third_string]  += str(note['fret']).ljust(note['time'], '-') if note['string'] == 3 else '-'.ljust(note['time'], '-')
    ascii_strings[fourth_string] += str(note['fret']).ljust(note['time'], '-') if note['string'] == 4 else '-'.ljust(note['time'], '-')
    ascii_strings[fifth_string]  += str(note['fret']).ljust(note['time'], '-') if note['string'] == 5 else '-'.ljust(note['time'], '-')

  return ascii_strings

In [129]:
chords = generate_moveable_chords()

In [130]:
first_chord = chords['alpha_g5']
second_chord = chords['alpha_c6']
third_chord = chords['alpha_d6']
fourth_chord = first_chord

roll = 'T4I3M2T3I2M1'
roll += roll
roll += 'T5I2T3M1'
print(roll)

T4I3M2T3I2M1T4I3M2T3I2M1T5I2T3M1


In [131]:
measures = []
for chord in [first_chord, second_chord, third_chord, fourth_chord]:
  prototab = roll_on_chord(roll_pattern=roll, chord=chord)
  measures.append(prototab_to_ascii(prototab))

In [132]:
measures[0][0]

'|----------5-----------5-------5-'

In [133]:
lines = [
  ''.join([m[0] for m in measures]),
  ''.join([m[1] for m in measures]),
  ''.join([m[2] for m in measures]),
  ''.join([m[3] for m in measures]),
  ''.join([m[4] for m in measures]),
]

lines = [line + '|' for line in lines]

print('\n'.join(lines))

|----------5-----------5-------5-|----------10----------10------10|----------12----------12------12|----------5-----------5-------5-|
|----3---3-------3---3-----3-----|----8---8-------8---8-----8-----|----10--10------10--10----10----|----3---3-------3---3-----3-----|
|--4---4-------4---4---------4---|--9---9-------9---9---------9---|--11--11------11--11--------11--|--4---4-------4---4---------4---|
|5-----------5-------------------|10----------10------------------|12----------12------------------|5-----------5-------------------|
|------------------------0-------|------------------------0-------|------------------------0-------|------------------------0-------|


In [89]:
[i**2 for i in (3, 2, 1)]

[9, 4, 1]